In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 1.3MB 8.9MB/s 


In [ ]:
def PrintRun():
    

In [ ]:

##### 단계별 런
# * 베이스 : 원핫 -> 스케일러
# * 머니니스 생성 : 원핫 -> 머니니스 -> 스케일러
# * 가입나이 : 원핫 -> 머니니스 -> 가입나이 -> 스케일러
# * 금리 : 원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 스케일러
# * 임베딩1 : 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 스케일러
# * 임베딩2 : 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러

jobName = '201216_Train_split1'

# 단계1. 베이스 : 원핫 -> 스케일러
x, y = LibData.GetXY_s_o(jobName)
print('step1')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계2. 머니니스 생성 : 원핫 -> 머니니스 -> 스케일러
x, y = LibData.GetXY_o(jobName)
x = LibData.GetMoneyness(x)
scaler, x = LibUtils.ScaleStandard(x)
print('step2')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계3. : 원핫 -> 머니니스 -> 금리축소 -> 스케일러
x, y = LibData.GetXY_o(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
scaler, x = LibUtils.ScaleStandard(x)
print('step3')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계4 : 원핫 -> 머니니스 -> 금리축소 -> 가입나이 -> 스케일러
x, y = LibData.GetXY_o(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
x = LibData.GetAttainedAge(x)
scaler, x = LibUtils.ScaleStandard(x)
print('step4')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계5. 임베딩1 : 원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
x = LibData.GetAttainedAge(x)
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode'])
scaler, x = LibUtils.ScaleStandard(x)
print('step5')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
x = LibData.GetAttainedAge(x)
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode'])
x = LibEE.GetXIntersected(x, ['ElapsedMth','AttainedAge'], 'Inter_EM_AA')
x, emb = LibEE.GetXEmbApplied(3, x, y, ['Inter_EM_AA'])
scaler, x = LibUtils.ScaleStandard(x)
print('step6')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))

# 단계7. 변수선택
colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x = x[colsSelected]
print('step7')
print(LibML.GetPredCV(x,y, LibML.XGBoostDefault(), 5))
print(LibML.GetPredCV(x,y, LibML.RandomForestDefault(len(x2.columns)), 5))

# 단계8. 튜닝
print('step8')
print(LibML.GetPredCV(x,y, LibML.XGBoostTuned(), 5))
print(LibML.GetPredCV(x,y, LibML.RandomForestTuned(len(x2.columns)), 5))

LibUtils.ELoop()

[163.52, 0.1955, 0.9344]
[166.17, 0.1987, 0.9051]
[150.67, 0.1802, 0.945]
[159.53, 0.1908, 0.9148]
[136.46, 0.1632, 0.9587]
[148.8, 0.1779, 0.9268]
[119.84, 0.1433, 0.9688]
[123.1, 0.1472, 0.9498]
[113.9, 0.1362, 0.9717]
[123.5, 0.1477, 0.9487]
[120.49, 0.1441, 0.9669]
[135.02, 0.1614, 0.9397]
[109.34, 0.1307, 0.9745]
[121.24, 0.145, 0.9508]
[66.47, 0.0795, 0.9778]


In [10]:

jobName = '201216_Train_split1'

# 단계5. 임베딩1 : 원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetIrReduction(x)
x = LibData.GetAttainedAge(x)
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode'])
scaler, x = LibUtils.ScaleStandard(x)
print('step5')
print(LibML.GetScoresCV(x,y, LibML.RandomForestDefault(len(x.columns)), 5), LibML.GetScoresCV(x,y, LibML.XGBoostDefault(), 5))


step5
start learning 1 2021-01-17 18:01... [124.85, 0.1493, 0.9478]... end learning 2021-01-17 18:05
start learning 2 2021-01-17 18:05... [121.83, 0.1472, 0.9497]... end learning 2021-01-17 18:08
start learning 3 2021-01-17 18:08... [123.44, 0.1473, 0.9509]... end learning 2021-01-17 18:11
start learning 4 2021-01-17 18:11... [120.7, 0.1437, 0.9546]... end learning 2021-01-17 18:14
start learning 5 2021-01-17 18:14... [128.22, 0.1527, 0.941]... end learning 2021-01-17 18:18
start learning 1 2021-01-17 18:18... [116.82, 0.1413, 0.9704]... end learning 2021-01-17 18:21
start learning 2 2021-01-17 18:21... [115.93, 0.1396, 0.9678]... end learning 2021-01-17 18:25
start learning 3 2021-01-17 18:25... [112.01, 0.1335, 0.9721]... end learning 2021-01-17 18:28
start learning 4 2021-01-17 18:28... [116.05, 0.1374, 0.9709]... end learning 2021-01-17 18:32
start learning 5 2021-01-17 18:32... [114.25, 0.1359, 0.9716]... end learning 2021-01-17 18:35
[123.81, 0.148, 0.9488] [115.01, 0.1375, 0.970